在沙盒环境，构建期货三种组合

In [1]:
import os,pdb,itertools,copy,datetime
os.environ['ULTRON_DATA'] = 'keim'

In [2]:
import pandas as pd
import numpy as np
from ultron.env import *
from ultron.strategy.experimental.multiple_factor import MultipleFactor

/var/log/ultron/2022-09-27.log


In [3]:
enable_example_env()

2022-09-27 16:48:13,133 - [env.py:67] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/keim


#### 加载行情数据

In [4]:
market_data = pd.read_csv(os.path.join(g_project_data, 'market_data.csv'), index_col=0)
market_data['trade_date'] = pd.to_datetime(market_data['trade_date'])
market_data.head()

,trade_date,code,closePrice
0,2017-10-27,A,4435.665713
1,2017-10-27,AL,15449.929290
2,2017-10-27,BU,3302.356787
3,2017-10-27,C,2002.534660
4,2017-10-27,CF,20449.104349


#### 加载信号值

In [5]:
signal_data = pd.read_csv(os.path.join(g_project_data, 'signal.csv'), index_col=0)
signal_data['trade_date'] = pd.to_datetime(signal_data['trade_date'])
signal_data.head()

,trade_date,code,BM_MainFar_80D_25_5_5,BM_MainFar_80D_27_5_5,BM_MainFar_80D_23_5_5,BM_RecentFar_20D_5_1_5,BM_RecentFar_40D_9_2_3,BM_RecentFar_40D_3_1_3,BM_RecentFar_40D_11_2_3,BM_RecentFar_40D_13_1_3,...,T_DnVolatility_2_20D_25_1_4,T_DnVolatility_1_10D_21_1_5,T_DnVolatility_2_20D_27_1_4,WeightNetIntTotalChg5D_11_5_3,WeightNetIntTotalChg5D_13_5_3,WeightShortVolRelTotIntChg_9_4_4,inventory_5_3_5,inventory_10_1_5,inventory_15_4_5,profitratio_5_3_5
0,2017-11-01,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-11-01,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-11-01,BU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-11-01,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11-01,CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
columns = [col for col in signal_data.columns if col not in ['trade_date','code']]

#### 等权合并信号

In [7]:
mixed_signal = copy.deepcopy(signal_data)
mixed_signal['signal'] = np.nanmean(mixed_signal.set_index(
    ['trade_date','code'])[columns],axis=1)
mixed_signal = mixed_signal[['trade_date','code','signal']]
mixed_signal = mixed_signal.dropna(subset=['signal'])
mixed_signal.head()

,trade_date,code,signal
0,2017-11-01,A,0.0
1,2017-11-01,AL,1.0
2,2017-11-01,BU,-1.0
3,2017-11-01,C,1.0
4,2017-11-01,CF,1.0


#### 前置收益率计算

In [8]:
def prev_returs_impl(price_data, key, name):
    price_tb = price_data[key].unstack()
    price_tb.fillna(method='pad', inplace=True)
    return_tb = np.log(price_tb / price_tb.shift(1))
    return_tb = return_tb.replace([np.inf, -np.inf], np.nan)
    return_tb = return_tb.stack().reindex(price_data.index)
    return_tb.name = name
    return return_tb

#### 后置收益率计算

In [9]:
def next_returs_impl(price_data, key, name):
    price_tb = price_data[key].unstack()
    price_tb.fillna(method='pad', inplace=True)
    return_tb = np.log(price_tb.shift(-1) / price_tb)
    return_tb = return_tb.replace([np.inf, -np.inf], np.nan)
    return_tb = return_tb.stack().reindex(price_data.index)
    return_tb.name = name
    return return_tb

In [10]:
next_rets = next_returs_impl(
    market_data.set_index(['trade_date','code']),'closePrice','nxt1_ret').reset_index()
next_rets['trade_date'] = pd.to_datetime(next_rets['trade_date'])
next_rets.head()

,trade_date,code,nxt1_ret
0,2017-10-27,A,-0.000276
1,2017-10-27,AL,0.001222
2,2017-10-27,BU,0.019945
3,2017-10-27,C,-0.005420
4,2017-10-27,CF,0.001003


In [11]:
prev_rets = prev_returs_impl(
    market_data.set_index(['trade_date','code']),'closePrice','pev1_ret').reset_index()
prev_rets['trade_date'] = pd.to_datetime(prev_rets['trade_date'])
prev_rets.tail()

,trade_date,code,pev1_ret
37248,2022-06-22,TA,-0.020493
37249,2022-06-22,V,-0.035797
37250,2022-06-22,Y,-0.040992
37251,2022-06-22,ZC,0.000237
37252,2022-06-22,ZN,-0.002764


#### 加载波动率

In [12]:
volatility_data = pd.read_csv(os.path.join(g_project_data, 'volatility_data.csv'), index_col=0)
volatility_data['trade_date'] = pd.to_datetime(volatility_data['trade_date'])
volatility_data.head()

,trade_date,code,VolatilitySTD20D
0,2017-10-27,A,0.098059
1,2017-10-27,AL,0.149470
2,2017-10-27,BU,0.210969
3,2017-10-27,C,0.078883
4,2017-10-27,CF,0.105971


#### 等权组合

In [13]:
equal_data  = MultipleFactor(signal_data=signal_data, 
                    volatility_data=None, 
                    returns_data=None)
equal_res = equal_data.equal_run(columns=columns, equal=1)

2022-09-27 16:48:17,544 - [multiple_factor.py:227] - ultron - INFO - starting construction


#### 信号加权组合

In [14]:
weighted  = MultipleFactor(signal_data=signal_data, 
                    volatility_data=None, 
                    returns_data=None)
weighted_pos = weighted.equal_run(columns=columns, equal=0)
weighted_pos.head()

2022-09-27 16:48:28,135 - [multiple_factor.py:227] - ultron - INFO - starting construction


,trade_date,weight,code
0,2017-11-01,0.090909,MA
1,2017-11-01,0.090909,AL
2,2017-11-01,0.090909,C
3,2017-11-01,0.090909,CF
4,2017-11-01,0.090909,CS


#### 目标波动约束组合

In [15]:
volatility = MultipleFactor(signal_data=signal_data.dropna(),
                                   volatility_data=volatility_data.dropna(),
                                   returns_data=prev_rets.dropna().rename(columns={'pev1_ret':'nxt1_ret'}))
volatility_pos = volatility.weighted_run(
        columns=columns,
        default_volatility=0.06,
        volatility_name='VolatilitySTD20D')

2022-09-27 16:48:42,230 - [multiple_factor.py:145] - ultron - INFO - starting construction
2022-09-27 16:48:46,806 - [multiple_factor.py:152] - ultron - INFO - initialize weight
2022-09-27 16:48:55,897 - [multiple_factor.py:168] - ultron - INFO - volatility winsorize
2022-09-27 16:48:59,188 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2017-11-16 00:00:00, end_date: 2017-12-14 00:00:00
2022-09-27 16:48:59,213 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2017-11-17 00:00:00, end_date: 2017-12-15 00:00:00
2022-09-27 16:48:59,234 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2017-11-20 00:00:00, end_date: 2017-12-18 00:00:00
2022-09-27 16:48:59,256 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2017-11-21 00:00:00, end_date: 2017-12-19 00:00:00
2022-09-27 16:48:59,277 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2017-11-22 00:00:00, end_date: 2017-12-20 00:00:00
2022-09-27 16:48:59,298 - [multiple_factor.py:181] - ultron - INFO - beg

2022-09-27 16:49:00,492 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-02-12 00:00:00, end_date: 2018-03-19 00:00:00
2022-09-27 16:49:00,514 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-02-13 00:00:00, end_date: 2018-03-20 00:00:00
2022-09-27 16:49:00,535 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-02-14 00:00:00, end_date: 2018-03-21 00:00:00
2022-09-27 16:49:00,556 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-02-22 00:00:00, end_date: 2018-03-22 00:00:00
2022-09-27 16:49:00,577 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-02-23 00:00:00, end_date: 2018-03-23 00:00:00
2022-09-27 16:49:00,599 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-02-26 00:00:00, end_date: 2018-03-26 00:00:00
2022-09-27 16:49:00,620 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-02-27 00:00:00, end_date: 2018-03-27 00:00:00
2022-09-27 16:49:00,641 - [multiple_factor.py:181] - ultron - INFO - begin_d

2022-09-27 16:49:01,845 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-05-23 00:00:00, end_date: 2018-06-21 00:00:00
2022-09-27 16:49:01,867 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-05-24 00:00:00, end_date: 2018-06-22 00:00:00
2022-09-27 16:49:01,889 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-05-25 00:00:00, end_date: 2018-06-25 00:00:00
2022-09-27 16:49:01,910 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-05-28 00:00:00, end_date: 2018-06-26 00:00:00
2022-09-27 16:49:01,932 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-05-29 00:00:00, end_date: 2018-06-27 00:00:00
2022-09-27 16:49:01,954 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-05-30 00:00:00, end_date: 2018-06-28 00:00:00
2022-09-27 16:49:01,976 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-05-31 00:00:00, end_date: 2018-06-29 00:00:00
2022-09-27 16:49:01,998 - [multiple_factor.py:181] - ultron - INFO - begin_d

2022-09-27 16:49:03,219 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-08-21 00:00:00, end_date: 2018-09-18 00:00:00
2022-09-27 16:49:03,241 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-08-22 00:00:00, end_date: 2018-09-19 00:00:00
2022-09-27 16:49:03,263 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-08-23 00:00:00, end_date: 2018-09-20 00:00:00
2022-09-27 16:49:03,284 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-08-24 00:00:00, end_date: 2018-09-21 00:00:00
2022-09-27 16:49:03,306 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-08-27 00:00:00, end_date: 2018-09-25 00:00:00
2022-09-27 16:49:03,328 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-08-28 00:00:00, end_date: 2018-09-26 00:00:00
2022-09-27 16:49:03,350 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-08-29 00:00:00, end_date: 2018-09-27 00:00:00
2022-09-27 16:49:03,372 - [multiple_factor.py:181] - ultron - INFO - begin_d

2022-09-27 16:49:04,595 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-11-26 00:00:00, end_date: 2018-12-24 00:00:00
2022-09-27 16:49:04,617 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-11-27 00:00:00, end_date: 2018-12-25 00:00:00
2022-09-27 16:49:04,639 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-11-28 00:00:00, end_date: 2018-12-26 00:00:00
2022-09-27 16:49:04,661 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-11-29 00:00:00, end_date: 2018-12-27 00:00:00
2022-09-27 16:49:04,682 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-11-30 00:00:00, end_date: 2018-12-28 00:00:00
2022-09-27 16:49:04,704 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-12-03 00:00:00, end_date: 2019-01-02 00:00:00
2022-09-27 16:49:04,726 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2018-12-04 00:00:00, end_date: 2019-01-03 00:00:00
2022-09-27 16:49:04,747 - [multiple_factor.py:181] - ultron - INFO - begin_d

...

2022-09-27 16:49:17,297 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-03-29 00:00:00, end_date: 2021-04-27 00:00:00
2022-09-27 16:49:17,320 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-03-30 00:00:00, end_date: 2021-04-28 00:00:00
2022-09-27 16:49:17,342 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-03-31 00:00:00, end_date: 2021-04-29 00:00:00
2022-09-27 16:49:17,365 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-04-01 00:00:00, end_date: 2021-04-30 00:00:00
2022-09-27 16:49:17,387 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-04-02 00:00:00, end_date: 2021-05-06 00:00:00
2022-09-27 16:49:17,410 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-04-06 00:00:00, end_date: 2021-05-07 00:00:00
2022-09-27 16:49:17,432 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-04-07 00:00:00, end_date: 2021-05-10 00:00:00
2022-09-27 16:49:17,455 - [multiple_factor.py:181] - ultron - INFO - begin_d

2022-09-27 16:49:18,730 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-07-01 00:00:00, end_date: 2021-07-29 00:00:00
2022-09-27 16:49:18,753 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-07-02 00:00:00, end_date: 2021-07-30 00:00:00
2022-09-27 16:49:18,776 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-07-05 00:00:00, end_date: 2021-08-02 00:00:00
2022-09-27 16:49:18,799 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-07-06 00:00:00, end_date: 2021-08-03 00:00:00
2022-09-27 16:49:18,822 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-07-07 00:00:00, end_date: 2021-08-04 00:00:00
2022-09-27 16:49:18,845 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-07-08 00:00:00, end_date: 2021-08-05 00:00:00
2022-09-27 16:49:18,867 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-07-09 00:00:00, end_date: 2021-08-06 00:00:00
2022-09-27 16:49:18,890 - [multiple_factor.py:181] - ultron - INFO - begin_d

2022-09-27 16:49:20,171 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-09-30 00:00:00, end_date: 2021-11-04 00:00:00
2022-09-27 16:49:20,194 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-10-08 00:00:00, end_date: 2021-11-05 00:00:00
2022-09-27 16:49:20,217 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-10-11 00:00:00, end_date: 2021-11-08 00:00:00
2022-09-27 16:49:20,240 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-10-12 00:00:00, end_date: 2021-11-09 00:00:00
2022-09-27 16:49:20,262 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-10-13 00:00:00, end_date: 2021-11-10 00:00:00
2022-09-27 16:49:20,285 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-10-14 00:00:00, end_date: 2021-11-11 00:00:00
2022-09-27 16:49:20,308 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2021-10-15 00:00:00, end_date: 2021-11-12 00:00:00
2022-09-27 16:49:20,331 - [multiple_factor.py:181] - ultron - INFO - begin_d

2022-09-27 16:49:21,605 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-01-05 00:00:00, end_date: 2022-02-09 00:00:00
2022-09-27 16:49:21,628 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-01-06 00:00:00, end_date: 2022-02-10 00:00:00
2022-09-27 16:49:21,651 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-01-07 00:00:00, end_date: 2022-02-11 00:00:00
2022-09-27 16:49:21,674 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-01-10 00:00:00, end_date: 2022-02-14 00:00:00
2022-09-27 16:49:21,697 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-01-11 00:00:00, end_date: 2022-02-15 00:00:00
2022-09-27 16:49:21,719 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-01-12 00:00:00, end_date: 2022-02-16 00:00:00
2022-09-27 16:49:21,742 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-01-13 00:00:00, end_date: 2022-02-17 00:00:00
2022-09-27 16:49:21,765 - [multiple_factor.py:181] - ultron - INFO - begin_d

2022-09-27 16:49:23,040 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-04-13 00:00:00, end_date: 2022-05-16 00:00:00
2022-09-27 16:49:23,063 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-04-14 00:00:00, end_date: 2022-05-17 00:00:00
2022-09-27 16:49:23,085 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-04-15 00:00:00, end_date: 2022-05-18 00:00:00
2022-09-27 16:49:23,108 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-04-18 00:00:00, end_date: 2022-05-19 00:00:00
2022-09-27 16:49:23,131 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-04-19 00:00:00, end_date: 2022-05-20 00:00:00
2022-09-27 16:49:23,153 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-04-20 00:00:00, end_date: 2022-05-23 00:00:00
2022-09-27 16:49:23,176 - [multiple_factor.py:181] - ultron - INFO - begin_date: 2022-04-21 00:00:00, end_date: 2022-05-24 00:00:00
2022-09-27 16:49:23,199 - [multiple_factor.py:181] - ultron - INFO - begin_d